In [ ]:
import pandas as pd
import itertools
import pulp


In [ ]:
df = pd.read_csv('filtered3_df_full.csv')
pred = pd.read_csv('filtered3_prediction.csv')
pred = pred[['PLAYER_NAME', 'PRED_SCORE']]
df.columns.values
df = df[['Season','Game_ID', 'PLAYER_NAME', 'POS', 'Team', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT',
       'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK',
       'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'VIDEO_AVAILABLE', 'DD', 'TD',
       'total_fantasy_points', 'salary']]


In [ ]:
# DO NOT RUN
def get_recent_salary(df):
    # sort by game date in descending order
    df_sorted = df.sort_values(by=['GAME_DATE'], ascending=False)
    
    # group by player name and get first row of each group to get player's most recent game 
    df_recent_salary = df_sorted.groupby('PLAYER_NAME').first()
    
    return df_recent_salary[['salary']].reset_index()

df_recent_salary = get_recent_salary(df)




In [ ]:
# Example lineup between game dates 2021-1-16 and 2021-1-17
# DO NOT RUN
df_ex = df[(df['GAME_DATE'] >= '2021-01-16') & (df['GAME_DATE'] <= '2021-01-30')]
merged_df = pd.merge(pred, df_recent_salary, on='PLAYER_NAME')
merged_df = pd.merge(merged_df, df_ex[['PLAYER_NAME', 'POS']], on='PLAYER_NAME')
merged_df

In [ ]:
# Example lineup between game dates 2021-1-16 and 2021-1-17
# RUN
df_ex = df[(df['GAME_DATE'] >= '2021-01-16') & (df['GAME_DATE'] <= '2021-01-17')]
merged_df = pd.merge(pred, df_ex[['PLAYER_NAME', 'salary', 'POS']], on='PLAYER_NAME')
merged_df
merged_df.columns.values
merged_df = merged_df.drop_duplicates(subset=['PLAYER_NAME'], keep='first')
merged_df


In [ ]:
filtered_df = merged_df[merged_df['POS'] == 'PG']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for PG: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'SG']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for SG: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'SF']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for SF: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'PF']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for PF: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)
filtered_df = merged_df[merged_df['POS'] == 'C']
min_value = filtered_df['salary'].min()
print(f"Minimum salary for C: {min_value}")
least_salary_rows = filtered_df.nsmallest(5, 'salary')
print(least_salary_rows)


In [ ]:
pg_rows = merged_df[merged_df['POS'] == 'PG']
pg_rows


In [ ]:
# WITHOUT SALARY CONSTRAINT

def optimize_lineup(merged_df):
    # filter by positions
    PG = merged_df[merged_df['POS'] == 'PG']
    SG = merged_df[merged_df['POS'] == 'SG']
    SF = merged_df[merged_df['POS'] == 'SF']
    PF = merged_df[merged_df['POS'] == 'PF']
    C = merged_df[merged_df['POS'] == 'C']

    # define positions and # of players for each position
    positions = {'PG': 1, 'SG': 1, 'SF': 1, 'PF': 1, 'C': 1, 'G': 1, 'F': 1, 'Util': 1}

    # initialize the lineup dictionary
    lineup = {position: [] for position in positions.keys()}

    # loop through the sorted dataframes for each position
    for pos, df in {'PG': PG, 'SG': SG, 'SF': SF, 'PF': PF, 'C': C}.items():
        df = df.sort_values(by='PRED_SCORE', ascending=False)

        for idx, player in df.iterrows():
            if positions[pos] > 0 and sum(positions.values()) > 1:
                lineup[pos].append(player['PLAYER_NAME'])
                positions[pos] -= 1

    # fill the G, F, and Util spots
    all_positions_df = merged_df.sort_values(by='PRED_SCORE', ascending=False)
    for idx, player in all_positions_df.iterrows():
        pos = player['POS']

        if positions['G'] > 0 and pos in {'PG', 'SG'} and player['PLAYER_NAME'] not in lineup['PG'] + lineup['SG']:
            lineup['G'].append(player['PLAYER_NAME'])
            positions['G'] -= 1
        elif positions['F'] > 0 and pos in {'SF', 'PF'} and player['PLAYER_NAME'] not in lineup['SF'] + lineup['PF']:
            lineup['F'].append(player['PLAYER_NAME'])
            positions['F'] -= 1
        elif positions['Util'] > 0 and player['PLAYER_NAME'] not in (lineup['PG'] + lineup['SG'] + lineup['SF'] +
                                                                     lineup['PF'] + lineup['C'] + lineup['G'] +
                                                                     lineup['F']):
            lineup['Util'].append(player['PLAYER_NAME'])
            positions['Util'] -= 1

        if all(count == 0 for count in positions.values()):
            break

    total_score = sum(merged_df.loc[merged_df['PLAYER_NAME'].isin(lineup[pos]), 'PRED_SCORE'].sum() for pos in lineup.keys())
    total_salary = sum(merged_df.loc[merged_df['PLAYER_NAME'].isin(lineup[pos]), 'salary'].sum() for pos in lineup.keys())

    # print the lineup and the total score and total salary
    print('Lineup:')
    for position, players in lineup.items():
        print(f'{position}: {", ".join(players)}')
    print(f'Total score: {total_score}')
    print(f'Total salary: {total_salary}')

optimize_lineup(merged_df)


In [ ]:
# DO NOT RUN THIS SHIT WITHOUT SERIOUS CAUTION 
# WITH SALARY CONSTRAINT

def optimize_lineup(merged_df, salary_cap=50000):
    # Filter players based on positions
    positions_data = {
        'PG': merged_df[merged_df['POS'] == 'PG'],
        'SG': merged_df[merged_df['POS'] == 'SG'],
        'SF': merged_df[merged_df['POS'] == 'SF'],
        'PF': merged_df[merged_df['POS'] == 'PF'],
        'C': merged_df[merged_df['POS'] == 'C']
    }
    
    # Generate all valid lineups
    valid_lineups = []
    
    for pg, sg, sf, pf, c in itertools.product(positions_data['PG'].iterrows(), positions_data['SG'].iterrows(),
                                               positions_data['SF'].iterrows(), positions_data['PF'].iterrows(),
                                               positions_data['C'].iterrows()):
        lineup = {'PG': pg[1], 'SG': sg[1], 'SF': sf[1], 'PF': pf[1], 'C': c[1], 'G': None, 'F': None, 'Util': None}
        remaining_players = merged_df.drop([pg[0], sg[0], sf[0], pf[0], c[0]])
        
        # Fill the G, F, and Util spots
        g_candidates = remaining_players[remaining_players['POS'].isin(['PG', 'SG'])]
        f_candidates = remaining_players[remaining_players['POS'].isin(['SF', 'PF'])]
        
        for g, f, util in itertools.product(g_candidates.iterrows(), f_candidates.iterrows(),
                                            remaining_players.iterrows()):
            if g[0] != f[0] and g[0] != util[0] and f[0] != util[0]:
                lineup['G'] = g[1]
                lineup['F'] = f[1]
                lineup['Util'] = util[1]
                
                total_salary = sum(player['salary'] for player in lineup.values())
                
                if total_salary <= salary_cap:
                    valid_lineups.append(lineup)
                    print('Valid Lineup:')
                    for position, player in lineup.items():
                        print(f'{position}: {player["PLAYER_NAME"]}')
                    print(f'Total salary: {total_salary}\n')
                    
                    
    # Find the optimal lineup
    if valid_lineups:
        optimal_lineup = max(valid_lineups, key=lambda l: sum(player['PRED_SCORE'] for player in l.values()))
        total_score = sum(player['PRED_SCORE'] for player in optimal_lineup.values())
        total_salary = sum(player['salary'] for player in optimal_lineup.values())
        
        print('Optimal Lineup:')
        for position, player in optimal_lineup.items():
            print(f'{position}: {player["PLAYER_NAME"]}')
        print(f'Total score: {total_score}')
        print(f'Total salary: {total_salary}')
    else:
        print('No valid lineup found within the salary constraint.')

optimize_lineup(merged_df)
